# Experiment Tracking

Format of the input: **[batch_size, no_of_channels, height, width]**

---

In [ ]:
# Check GPU information
!nvidia-smi

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
import scripts.data_setup as data_setup
import scripts.engine as engine


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
RANDOM_SEED = 42

In [ ]:
print(torch.__version__)
print(torchvision.__version__)

## 1. Get data ready (turn into tensor)
Our dataset is a subset of the Food101 dataset. Food101 starts 101 different classes of food and 1000 images per class (750 training, 250 testing). Our dataset starts with 3 classes of food and only 10% of the images (~75 training, 25 testing).

Why do this?
- When starting out ML projects, it's important to try things on a small scale and then increase the scale when necessary.
- The whole point is to speed up how fast you can experiment.

### 1.1 Get the data

In [ ]:
import scripts.get_data as get_data
import os


url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
image_path = os.path.join(os.getcwd(), "data", "pizza_steak_sushi")
get_data.download_data(url=url, save_path_str=image_path)

In [ ]:
# Check details about data

import os
def walk_through_dir(dir_path):
    """Walks through dir_path returning its contents."""
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

walk_through_dir(image_path)

In [ ]:
from pathlib import Path


# Setup train and testing paths
image_path = Path(image_path)
train_dir = image_path / "train"
test_dir = image_path / "test"

In [ ]:
# Visualize the data (randomly take some images)

import random 
from PIL import Image

# Set seed
# random.seed(RANDOM_SEED)

# 1. Get all image paths 
image_path_list = list(image_path.glob("*/*/*.jpg"))

# 2. Pick a random image path
random_image_path = random.choice(image_path_list)

# 3. Get image class from path name (the image class is the name of the directory where the image is stored)
image_class = random_image_path.parent.stem

# 4. Open image
img = Image.open(random_image_path)

# 5. Print metadata 
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Turn the image into an array
img_as_array = np.asarray(img)

# Plot the image with matplotlib
plt.figure(figsize=(10, 7))
plt.imshow(img_as_array)
plt.title(f"Image class: {image_class} | Image shape: {img_as_array.shape} -> [height, width, color_channels] (HWC)")
plt.axis(False)

### 1.2 Transform the data
It's important that your custom data going into the model is prepared in the same way as the original training data that went into the model. Two ways to do it:
1. Manually create the same transformation as used in the pre-trained model.
2. Automatically get the transofrmation used in the pre-trained model.
    - `EfficientNet_B0_Weights` is the model architecture weights we'd like to use 

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


# Create a transforms pipeline manually (required for torchvision < 0.13)
manual_transform = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape all images to 224x224 (though some models may require different sizes)
    transforms.ToTensor(), # 2. Turn image values to between 0 & 1 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. A mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std=[0.229, 0.224, 0.225]) # 4. A standard deviation of [0.229, 0.224, 0.225] (across each colour channel),
])

# Get transforms from pre-trained model (required for torchvision >= 0.13)
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights from pretraining on ImageNet
auto_transform = weights.transforms()

print(f"Shape of image before transform: {img_as_array.shape}")
print(f"Shape of image after manual transform: {manual_transform(img).shape}")
print(f"Shape of image after auto transform: {auto_transform(img).shape}")

In [ ]:
def plot_transformed_images(image_paths: list, transform, n=3, seed=None):
  """
  Selects random images from a path of images and loads/transforms 
  them then plots the original vs the transformed version.
  """
  if seed:
    random.seed(seed)
  random_image_paths = random.sample(image_paths, k=n)
  for image_path in random_image_paths:
    with Image.open(image_path) as f:
      fig, ax = plt.subplots(nrows=1, ncols=2)
      ax[0].imshow(f)
      ax[0].set_title(f"Original\nSize: {f.size}")
      ax[0].axis(False)

      # Transform and plot target image
      transformed_image = transform(f).permute(1, 2, 0) # note we will need to change shape for matplotlib (C, H, W) -> (H, W, C)
      ax[1].imshow(transformed_image)
      ax[1].set_title(f"Transformed\nShape: {transformed_image.shape}")
      ax[1].axis("off")

      fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)

plot_transformed_images(image_paths=image_path_list,
                        transform=auto_transform,
                        n=3,
                        seed=None)

### 1.3 Convert data to pytorch dataset
We'll do this in next section.

### 1.4 Prepare DataLoader

In [ ]:
from scripts.data_setup import create_dataloaders


# Create training and testing DataLoaders as well as get a list of class names
train_dataloader, test_dataloader, class_names = create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=auto_transform, # resize, convert images to between 0 & 1 and normalize them
    batch_size=32) # set mini-batch size to 32

train_dataloader, test_dataloader, class_names

In [ ]:
# Visualize one sample from train_dataloader
import numpy as np

train_features_batch, train_labels_batch = next(iter(train_dataloader))
print(f"train_features_batch: {train_features_batch.shape}")
print(f"train_labels_batch: {train_labels_batch.shape}")

torch.manual_seed(RANDOM_SEED)
rand_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[rand_idx], train_labels_batch[rand_idx]

img_as_array = np.asarray(img.permute(1, 2, 0))
plt.imshow(img_as_array)
plt.title(f"{label}: {class_names[label]}")
plt.axis(False)
print(f"Image size: {img.shape}")
print(f"Label size: {label.shape}")

## 2. Build or pick a pretrained model for training

When starting to build a series of machine learning modelling experiments, it's best practice to start with a baseline model. A baseline model is a simple model you will try and improve upon with subsequent models/experiments. 

### 2.0 Pre-trained model

#### 2.0.1 Pick a pre-trained model


In [ ]:
# OLD: Setup the model with pretrained weights and send it to the target device (this was prior to torchvision v0.13)
# model = torchvision.models.efficientnet_b0(pretrained=True).to(device) # OLD method (with pretrained=True)

# NEW: Setup the model with pretrained weights and send it to the target device (torchvision v0.13+)
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights 
model = torchvision.models.efficientnet_b0(weights=weights).to(DEVICE)

# model # uncomment to output (it's very long)

#### 2.0.2 Summary of model using torchinfo

In [ ]:
from torchinfo import summary

# Print a summary using torchinfo (uncomment for actual output)
summary(model=model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

#### 2.0.3 Freezing the base model and changing the output layer to suit our needs
To freeze layers means to keep them how they are during training. For example, if your model has pretrained layers, to freeze them would be to say, "don't change any of the patterns in these layers during training, keep them how they are." In essence, we'd like to keep the pretrained weights/patterns our model has learned from ImageNet as a backbone and then only change the output layers.

In [ ]:
# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in model.features.parameters():
    param.requires_grad = False

In [ ]:
# Change the output layer

# Set the manual seeds
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)

# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)

# Recreate the classifier layer and seed it to the target device
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=1280, 
                    out_features=output_shape, # same number of output units as our number of classes
                    bias=True)).to(DEVICE)

In [ ]:
# # Do a summary *after* freezing the features and changing the output classifier layer (uncomment for actual output)
summary(model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

### 2.1 Pick loss function and optimizer

In [ ]:
import torch.nn as nn


loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

### 2.2 Build a training loop to train the model

In [ ]:
from timeit import default_timer as timer
from scripts.engine import train
from scripts.utils import print_train_time, plot_loss_curves, create_writer


torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)

start_timer = timer()
# Setup training and save the results
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5,
                device=DEVICE,
                writer=create_writer("data_10_percent", "effnetb0", "5_epochs"))
end_timer = timer()
train_time_0 = print_train_time(start_timer, end_timer, DEVICE)

print(results)
plot_loss_curves(results)

### 2.3 View model's results in TensorBoard

In [ ]:
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir runs

## 3. Make prediction
One thing we'll have to remember is that for our model to make predictions on an image, the image has to be in same format as the images our model was trained on.

This means we'll need to make sure our images have:
- Same shape - If our images are different shapes to what our model was trained on, we'll get shape errors.
- Same datatype - If our images are a different datatype (e.g. torch.int8 vs. torch.float32) we'll get datatype errors.
- Same device - If our images are on a different device to our model, we'll get device errors.
- Same transformations - If our model is trained on images that have been transformed in certain way (e.g. normalized with a specific mean and standard deviation) and we try and make preidctions on images transformed in a different way, these predictions may be off.

In [ ]:
from scripts.utils import pred_and_plot_single_image
from pathlib import Path

# Get a random list of image paths from test set
import random
num_images_to_plot = 3
test_image_path_list = list(Path(test_dir).glob("*/*.jpg")) # get list all image paths from test data 
test_image_path_sample = random.sample(population=test_image_path_list, # go through all of the test image paths
                                       k=num_images_to_plot) # randomly select 'k' image paths to pred and plot

# Make predictions on and plot the images
for image_path in test_image_path_sample:
    pred_and_plot_single_image(
        model=model, 
        image_path=image_path,
        class_names=class_names,
        transform=weights.transforms(), # optionally pass in a specified transform from our pretrained model weights
        # image_size=(224, 224)
    )

## 4. Evaluate the model

In [ ]:
from sklearn.metrics import accuracy_score
from scripts.utils import eval_model_classification


torch.manual_seed(RANDOM_SEED)
model_0_res = eval_model_classification(
    model=model,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_score
)
print(model_0_res)

In [ ]:
# Visualize random predictions
from scripts.utils import make_predictions

# random.seed(RANDOM_SEED)
y_preds = make_predictions(model, test_dataloader)

test_data = test_dataloader.dataset
rows, cols = 3, 3
fig = plt.figure(figsize=(9, 9))
for i in range(1, rows * cols + 1):
  rand_idx = torch.randint(0, len(test_data), size=[1]).item()
  X, y_truth = test_data[rand_idx]
  y_truth = class_names[y_truth]
  y_pred = class_names[y_preds[rand_idx]]
  fig.add_subplot(rows, cols, i)
  img_as_array = np.asarray(X.permute(1, 2, 0))
  plt.imshow(img_as_array)
  if y_pred == y_truth:
    plt.title(f"Truth: {y_truth} | Pred: {y_pred}", c="g")
  else:
    plt.title(f"Truth: {y_truth} | Pred: {y_pred}", c="r")
  plt.axis(False)

In [ ]:
from scripts.utils import make_predictions


# Make predictions with trained model
y_pred_tensor = make_predictions(model, test_dataloader, DEVICE)
y_pred_tensor

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

test_truth = torch.cat([y for X, y in test_dataloader])

# 2. Setup confusion instance and compare predictions to targets
confmat = ConfusionMatrix(task='multiclass', num_classes=len(class_names))
confmat_tensor = confmat(preds=y_pred_tensor,
                         target=test_truth)

# 3. Plot the confusion matrix
fig, ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(), # matplotlib likes working with numpy
    class_names=class_names,
    figsize=(10, 7)
)

## 5. Improve through experimentation
Let's try out the following:

| Experiment number | Training Dataset | Model (pretrained on ImageNet) | Number of epochs |
| --- | --- | --- | --- |
| 1	| Pizza, Steak, Sushi 10% percent | EfficientNetB0 | 5 |
| 2	| Pizza, Steak, Sushi 10% percent | EfficientNetB2 | 5 |
| 3	| Pizza, Steak, Sushi 10% percent | EfficientNetB0 | 10 |
| 4	| Pizza, Steak, Sushi 10% percent | EfficientNetB2 | 10 |
| 5	| Pizza, Steak, Sushi 20% percent | EfficientNetB0 | 5 |
| 6	| Pizza, Steak, Sushi 20% percent | EfficientNetB2 | 5 |
| 7	| Pizza, Steak, Sushi 20% percent | EfficientNetB0 | 10 |
| 8	| Pizza, Steak, Sushi 20% percent | EfficientNetB2 | 10 |

Notice how we're slowly scaling things up. With each experiment we slowly increase the amount of data, the model size and the length of training. By the end, experiment 8 will be using double the data, double the model size and double the length of training compared to experiment 1.

### 5.1 Download required datasets

In [ ]:
from scripts.get_data import download_data

# Download 10 percent and 20 percent training data (if necessary)
data_10_percent_path = download_data(url="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     save_path_str="pizza_steak_sushi")

data_20_percent_path = download_data(url="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     save_path_str="pizza_steak_sushi_20_percent")

In [ ]:
# Setup training directory paths
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

# Setup testing directory paths (note: use the same test dataset for both to compare the results)
test_dir = data_10_percent_path / "test"

# Check the directories
print(f"Training directory 10%: {train_dir_10_percent}")
print(f"Training directory 20%: {train_dir_20_percent}")
print(f"Testing directory: {test_dir}")

### 5.2 Transform Datasets and create DataLoaders

In [ ]:
from torchvision import transforms

# Create a transform to normalize data distribution to be inline with ImageNet
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], # values per colour channel [red, green, blue]
                                 std=[0.229, 0.224, 0.225]) # values per colour channel [red, green, blue]

# Compose transforms into a pipeline
simple_transform = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Resize the images
    transforms.ToTensor(), # 2. Turn the images into tensors with values between 0 & 1
    normalize # 3. Normalize the images so their distributions match the ImageNet dataset 
])

In [ ]:
BATCH_SIZE = 32

# Create 10% training and test DataLoaders
train_dataloader_10_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_10_percent,
    test_dir=test_dir, 
    transform=simple_transform,
    batch_size=BATCH_SIZE
)

# Create 20% training and test data DataLoders
train_dataloader_20_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_20_percent,
    test_dir=test_dir,
    transform=simple_transform,
    batch_size=BATCH_SIZE
)

# Find the number of samples/batches per dataloader (using the same test_dataloader for both experiments)
print(f"Number of batches of size {BATCH_SIZE} in 10 percent training data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 20 percent training data: {len(train_dataloader_20_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in testing data: {len(train_dataloader_10_percent)} (all experiments will use the same test set)")
print(f"Number of classes: {len(class_names)}, class names: {class_names}")

### 5.3 Create feature extraction models

> Note: Whenever you use a different model, one of the first things you should inspect is the input and output shapes. That way you'll know how you'll have to prepare your input data/update the model to have the correct output shape.

In [ ]:
import torchvision
from torchinfo import summary

# 1. Create an instance of EffNetB2 with pretrained weights
effnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT # "DEFAULT" means best available weights
effnetb2 = torchvision.models.efficientnet_b2(weights=effnetb2_weights)

# # 2. Get a summary of standard EffNetB2 from torchvision.models (uncomment for full output)
# summary(model=effnetb2, 
#         input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
#         # col_names=["input_size"], # uncomment for smaller output
#         col_names=["input_size", "output_size", "num_params", "trainable"],
#         col_width=20,
#         row_settings=["var_names"]
# ) 

# 3. Get the number of in_features of the EfficientNetB2 classifier layer
print(f"Number of in_features to final layer of EfficientNetB2: {len(effnetb2.classifier.state_dict()['1.weight'][0])}")

In [ ]:
import torchvision
from torch import nn
from scripts.utils import set_seeds

# Get num out features (one for each class pizza, steak, sushi)
OUT_FEATURES = len(class_names)

# Create an EffNetB0 feature extractor
def create_effnetb0():
    # 1. Get the base mdoel with pretrained weights and send to target device
    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    model = torchvision.models.efficientnet_b0(weights=weights).to(DEVICE)

    # 2. Freeze the base model layers
    for param in model.features.parameters():
        param.requires_grad = False

    # 3. Set the seeds
    set_seeds()

    # 4. Change the classifier head
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(in_features=1280, out_features=OUT_FEATURES)
    ).to(DEVICE)

    # 5. Give the model a name
    model.name = "effnetb0"
    print(f"[INFO] Created new {model.name} model.")
    return model

# Create an EffNetB2 feature extractor
def create_effnetb2():
    # 1. Get the base model with pretrained weights and send to target device
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    model = torchvision.models.efficientnet_b2(weights=weights).to(DEVICE)

    # 2. Freeze the base model layers
    for param in model.features.parameters():
        param.requires_grad = False

    # 3. Set the seeds
    set_seeds()

    # 4. Change the classifier head
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1408, out_features=OUT_FEATURES)
    ).to(DEVICE)

    # 5. Give the model a name
    model.name = "effnetb2"
    print(f"[INFO] Created new {model.name} model.")
    return model

### 5.4 Create experiments and set up training code

In [ ]:
# 1. Create epochs list
num_epochs = [5, 10]

# 2. Create models list (need to create a new model for each experiment)
models = ["effnetb0", "effnetb2"]

# 3. Create dataloaders dictionary for various dataloaders
train_dataloaders = {"data_10_percent": train_dataloader_10_percent,
                     "data_20_percent": train_dataloader_20_percent}

In [ ]:
%%time
from scripts.utils import save_model, create_writer
from scripts.engine import train

# 1. Set the random seeds
set_seeds(seed=42)

# 2. Keep track of experiment numbers
experiment_number = 0

# 3. Loop through each DataLoader
for dataloader_name, train_dataloader in train_dataloaders.items():

    # 4. Loop through each number of epochs
    for epochs in num_epochs: 

        # 5. Loop through each model name and create a new model based on the name
        for model_name in models:

            # 6. Create information print outs
            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")  

            # 7. Select the model
            if model_name == "effnetb0":
                model = create_effnetb0() # creates a new model each time (important because we want each experiment to start from scratch)
            else:
                model = create_effnetb2() # creates a new model each time (important because we want each experiment to start from scratch)
            
            # 8. Create a new loss and optimizer for every model
            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

            # 9. Train target model with target dataloaders and track experiments
            train(model=model,
                  train_dataloader=train_dataloader,
                  test_dataloader=test_dataloader, 
                  optimizer=optimizer,
                  loss_fn=loss_fn,
                  epochs=epochs,
                  device=DEVICE,
                  writer=create_writer(experiment_name=dataloader_name,
                                       model_name=model_name,
                                       extra=f"{epochs}_epochs"))
            
            # 10. Save the model to file so we can get back the best model
            save_filepath = f"06_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            save_model(model=model,
                       target_dir="models",
                       model_name=save_filepath)
            print("-"*50 + "\n")

### 5.5 View experiments in TensorBoard

In [ ]:
# Viewing TensorBoard in Jupyter and Google Colab Notebooks (uncomment to view full TensorBoard instance)
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# ---------------------- DO NOT TRY THIS UNLESS REQUIRED ---------------------
# # Upload the results to TensorBoard.dev (uncomment to try it out)
# !tensorboard dev upload --logdir runs \
#     --name "07. PyTorch Experiment Tracking: FoodVision Mini model results" \
#     --description "Comparing results of different model size, training data amount and training time."

## 6. Save and reload best trained model

In [ ]:
# Model has already been saved

In [ ]:
# Load the model
import os

# Setup the best model filepath
best_model_path = os.path.join(os.getcwd(), "models", "06_effnetb2_data_20_percent_10_epochs.pth")

# Instantiate a new instance of EffNetB2 (to load the saved state_dict() to)
best_model = create_effnetb2()

# Load the saved best model state_dict()
best_model.load_state_dict(torch.load(best_model_path))

In [ ]:
# Check the model file size
from pathlib import Path

# Get the model size in bytes then convert to megabytes
effnetb2_model_size = Path(best_model_path).stat().st_size // (1024*1024)
print(f"EfficientNetB2 feature extractor model size: {effnetb2_model_size} MB")

In [ ]:
# Import function to make predictions on images and plot them 
from scripts.utils import pred_and_plot_single_image

# Get a random list of 3 images from 20% test set
import random
num_images_to_plot = 3
test_image_path_list = list(Path(data_20_percent_path / "test").glob("*/*.jpg")) # get all test image paths from 20% dataset
test_image_path_sample = random.sample(population=test_image_path_list,
                                       k=num_images_to_plot) # randomly select k number of images

# Iterate through random test image paths, make predictions on them and plot them
for image_path in test_image_path_sample:
    pred_and_plot_single_image(
        model=best_model,
        image_path=image_path,
        class_names=class_names,
        transform=torchvision.models.EfficientNet_B0_Weights.DEFAULT.transforms()
        # image_size=(224, 224)
    )

In [ ]:
# Evaluate loaded model
from scripts.utils import eval_model_classification
from sklearn.metrics import accuracy_score


torch.manual_seed(RANDOM_SEED)

best_model_results = eval_model_classification(
    model=best_model,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_score
)

best_model_results

In [ ]:
from scripts.utils import make_predictions


# Make predictions with trained model
y_pred_tensor = make_predictions(best_model, test_dataloader, DEVICE)
y_pred_tensor

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

test_truth = torch.cat([y for X, y in test_dataloader])

# 2. Setup confusion instance and compare predictions to targets
confmat = ConfusionMatrix(task='multiclass', num_classes=len(class_names))
confmat_tensor = confmat(preds=y_pred_tensor,
                         target=test_truth)

# 3. Plot the confusion matrix
fig, ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(), # matplotlib likes working with numpy
    class_names=class_names,
    figsize=(10, 7)
)

## 7. Making prediciton on a custom image

In [ ]:
# Download custom image
import requests

# Setup custom image path
custom_image_path = image_path.parent / "04-pizza-dad.jpeg"

# Download the image if it doesn't already exist
if not custom_image_path.is_file():
  with open(custom_image_path, "wb") as f:
    # When downloading from GitHub, need to use the "raw" file link
    request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/04-pizza-dad.jpeg")
    print(f"Downloading {custom_image_path}...")
    f.write(request.content)
else:
  print(f"{custom_image_path} already exists, skipping download...")

In [ ]:
from scripts.utils import pred_and_plot_single_image

# Pred on our custom image
pred_and_plot_single_image(
    model=best_model,
    image_path=custom_image_path,
    class_names=class_names,
    transform=torchvision.models.EfficientNet_B0_Weights.DEFAULT.transforms(),
    device=DEVICE
)